Script to Clean up Lab Automation Data before Spotfire Import - 

Script 1 is for 1 single file (latest file)

Script 2 is for all unprocessed files in the folder

# Script 1

In [140]:

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pathlib


# * means all if need specific format then *.csv
list_of_files = glob.glob('C:/Datafiles/AMC1300/*.txt') 
latest_file = max(list_of_files, key=os.path.getctime)
print (latest_file)
File_name=os.path.basename(latest_file)
print(File_name)
#Remove the .txt extension
File_name = File_name[:-4]

path = pathlib.Path('C:/Datafiles/AMC1300/Cleanedup/'+File_name+'.csv')
re_search = re.search('..Temporary..', File_name)


 
if (path.is_file() == False and re_search == None):
#Read the file
        print("Inside the IF")
        df = pd.read_csv(latest_file,sep='\t',delimiter='\t')
        df=df.set_index('Meas_Name')
        df=df.drop(['RunComment','STRING','TEST_ID'])
        df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Start time','Lot','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        df.to_csv('C:/Datafiles/AMC1300/Cleanedup/'+File_name+'.csv')
     

C:/Datafiles/AMC1300\AMC1311_161558551- Temporary Data File.txt
AMC1311_161558551- Temporary Data File.txt


# Script 2 - Removes trash columns from data log and inserts type row


In [8]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pathlib
import re
list_of_files_raw_noext=[]
list_of_files_raw = glob.glob('C:/Datafiles/AMC1300/*.txt') 
No_of_files_processed=0
No_of_files_skipped=0
PATH_FOR_CLEAN_DATALOG= 'C:/Datafiles/AMC1300/Cleanedup/'
PATH_FOR_RAW_DATALOG= 'C:/Datafiles/AMC1300/'

for file in list_of_files_raw:
    re_search_bak = re.search('..bak', file)
    
    file_noext =os.path.basename(file[:-4])
    list_of_files_raw_noext.append(file)
    path = pathlib.Path(PATH_FOR_CLEAN_DATALOG+file_noext+'.csv')
    
    re_search = re.search('..Temporary..', file_noext)
    
    if (path.is_file() == False and re_search==None and re_search_bak==None):
        print("Processing file..."+os.path.basename(file))
         
        #Read the file
        df = pd.read_csv(PATH_FOR_RAW_DATALOG+file_noext+'.txt',sep='\t',delimiter='\t')
        
        #set Meas_Name as the DF index
        df=df.set_index('Device Descriptor')
        
        #Rows to be removed (dropped)
        df=df.drop(['DEV_ID','STRING'])
      
         #columns to be removed (dropped)

        df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Start time','Lot','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        df = df.rename(columns={col: col.split('(')[0] for col in df.columns})
        
        
        type_row=[]    
        for column in df:
            #print(column)
            ##print(df[column].unique())
            #print('Column index is',df.columns.get_loc(column))
            Column_idx=df.columns.get_loc(column)
            for unique_val in df[column].unique():
                if(pd.isnull(unique_val)==True):
                #print("this element is NaN")
                  pass
                else:
                    try:
                        float(unique_val)
                        isnumber= True
                        type_row.append(3.05) #just insert a real num in the first row so that spotfire detects it
                        #print(column ,'is a Number')
                        break 
                    except:
                        Isnumber = False   
                        type_row.append('STRING')
                        #print( column,'is a String')
                        break       
        
        
        #Make a data frame with df.columns as index and type_row as column
        df2 = pd.Series(type_row, index=[df.columns])
        #append df2 transposed to df, gets type_as the last row
        df2 = df.append(df2, ignore_index=True)
        #swap last row(type row ) and first row
        last_row, first_row = df2.iloc[-1].copy(), df2.iloc[0].copy()
        df2.iloc[-1],df2.iloc[0] = first_row,last_row
        
        df2.to_csv(PATH_FOR_CLEAN_DATALOG+file_noext+'.csv')







Processing file...AMC1300_Data.txt


C:\Users\a0406675\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,1,3,4,5,9,12,13,14,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,37,38,39,40,41,42,43,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# All code below is invalid and is used for debug

In [ ]:
 #columns to be removed (dropped)

df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Start time','Lot','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        df = df.rename(columns={col: col.split('(')[0] for col in df.columns})      
# Type row code
        
        # write cleaned up dataframe to .csv file of the same name
        df2.to_csv('C:/Datafiles/AMC1300/Cleanedup/'+file_noext+'.csv')
        print ('Saved as...'+file_noext+'.csv')
#         writer = pd.ExcelWriter('C:/Datafiles/AMC1300/Cleanedup/'+file_noext+'.xlsx')
#         df2.to_excel(writer,'Sheet1')
#         writer.save()
#         print ('Saved as...'+file_noext+'.xlsx')
        #update processed file counter
        No_of_files_processed+=1
       
    else :
        print ('Skipping file...',os.path.basename(file)) 
        No_of_files_skipped+=1     
            
            # import pdb; pdb.set_trace()     # Break point for debug
print(No_of_files_processed,'files processed')  
print (No_of_files_skipped,'files skipped')


#for file in list_of_files_processed:
 #   file =os.path.basename(file[:-4])
    #print(file)
    #list_of_files_processed_noext.append(file)



##for file in list_of_files_raw:
 ##   file =file[:-4]
    #print(file)
   ## list_of_files_raw_noext.append(file)
    
##for file in list_of_files_processed:
  ##  file =file[:-4]
    #print(file)
    ##list_of_files_processed_noext.append(file)    
    
##print(list_of_files_raw_noext)  
#print(list_of_files_processed_noext)   


#s = set(list_of_files_processed_noext)
#list_of_files_notprocessed_noext = [x for x in list_of_files_raw_noext if x not in s]
#print(list_of_files_notprocessed_noext)  




#Type Row Code


        
        #print(df.dtypes)
        
        type_row=[]    
        for column in df:
            #print(column)
            ##print(df[column].unique())
            #print('Column index is',df.columns.get_loc(column))
            Column_idx=df.columns.get_loc(column)
            for unique_val in df[column].unique():
                if(pd.isnull(unique_val)==True):
                #print("this element is NaN")
                  pass
                else:
                    try:
                        float(unique_val)
                        isnumber= True
                        type_row.append('FLOAT')
                        #print(column ,'is a Number')
                        break 
                    except:
                        Isnumber = False   
                        type_row.append('STRING')
                        #print( column,'is a String')
                        break       
        
        
        #Make a data frame with df.columns as index and type_row as column
        df2 = pd.Series(type_row, index=[df.columns])
        #append df2 transposed to df, gets type_as the last row
        df2 = df.append(df2, ignore_index=True)
        #swap last row(type row ) and first row
        last_row, first_row = df2.iloc[-1].copy(), df2.iloc[0].copy()
        df2.iloc[-1],df2.iloc[0] = first_row,last_row

